##### Grading Feedback
Final Grade: 68/70

# Question 0 (-2 If not answered)
Please provide the following the data so we can verify your github information and ensure accurate grading:
- Your Name: Chaithra Kopparam Cheluvaiah
- Your SU ID: 326926205

# IST 718: Big Data Analytics

- Professors: 
  - Willard Williamson <wewillia@syr.edu>
  - Emory Creel <emcreel@g.syr.edu>
- Faculty Assistants: 
  - Warren Justin Fernandes <wjfernan@syr.edu>
  - Ruchita Hiteshkumar Harsora <	rharsora@g.syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Code from the class text books or class provided code can be copied in its entirety.__
- Google Colab is the official class runtime environment so you should test your code on Colab before submission.
- Do not modify cells marked as grading cells or marked as do not modify.
- Before submitting your work, remember to check for run time errors with the following procedure:
`Runtime `$\rightarrow$ Factory reset runtime followed by Runtime $\rightarrow$ Run All.  All runtime errors will result in a minimum penalty of half off.
- All plots shall include descriptive title and axis labels.  Plot legends shall be included where possible.  Unless stated otherwise, plots can be made using any Python plotting package.
- Grading feedback cells are there for graders to provide feedback to students.  Don't change or remove grading feedback cells.
- Don't add or remove files from your git repo.
- Do not change file names in your repo.  This also means don't change the title of the ipython notebook.
- You are free to add additional code cells around the cells marked `your code here`.
- import * is not allowed because it is considered a very bad coding practice and in some cases can result in a significant delay (which slows down the grading process) in loading imports.  For example, the statement `from sympy import *` is not allowed.  You must import the specific packages that you need. 
- The graders reserve the right to deduct points for subjective things we see with your code.  For example, if we ask you to create a pandas data frame to display values from an investigation and you hard code the values, we will take points off for that.  This is only one of many different things we could find in reviewing your code.  In general, write your code like you are submitting it for a code peer review in industry.  
- Level of effort is part of our subjective grading.  For example, in cases where we ask for a more open ended investigation, some students put in significant effort and some students do the minimum possible to meet requirements.  In these cases, we may take points off for students who did not put in much effort as compared to students who put in a lot of effort.  We feel that the students who did a better job deserve a better grade.  We reserve the right to invoke level of effort grading at any time.
- Your notebook must run from start to finish without requiring manual input by the graders.  For example, do not mount your personal Google drive in your notebook as this will require graders to perform manual steps.  In short, your notebook should run from start to finish with no runtime errors and no need for graders to perform any manual steps.

I was very disappointed with the linear regression model accuracy releted to the insurance data set in homework 3.  I'm sure you were disappointed too.  In this homework, we will revisit the insurance data set and try to improve prediction scores.  Specifically, we will use random forest, gradient boosting trees, and deep learning to see if we can improve upon the scores achieved in homework 3.  Part 1 of the assignment will explore random forest and GBT.  Part 2 of the assignment will use deep learning.

In [1]:
# Grading Cell
enable_grid_search = False

The following cell is used to read the insurance data set into the colab environment.  Do not change or modify the following cell.

In [2]:
%%bash
# Do not change or modify this cell
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already installed
pip install pyspark &> /dev/null

# Download the data files from github
# If the data file does not exist in the colab environment
data_file_1=insurance.csv

if [[ ! -f ./${data_file_1} ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/wewilli1/ist718_data/master/${data_file_1} &> /dev/null
fi

In [3]:
#creating spark session and spark context
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ist718-hw06-ensemble').getOrCreate()
sc = spark.sparkContext

In [4]:
spark # checking the spark version

In [5]:
#loading the required libraries
import pyspark.sql.functions as f
from pyspark.ml import Pipeline, feature
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assignment Specific Instructions
Your grade for grid search problems in this assignment will be determined in part on level of effort and your model performance results as compared to other students in the class.

In this assignment, we will be comparing scores between random forest, gradient boosting trees, and deep learning.  You are required to correctly use train / test / validation data sets for model comparison as outlined in lecture.  Use train and test sets to train and score individual models during grid search.  Only use validation data to compare scores between models. You must name your data sets exactly `train`, `test`, and `validation` so that the graders know what data set is being used in each question.  I will be taking off 1 letter grade (10 points) for not following these instructions.

# Question 1 (10 pts)
- Read the insurance data file into a spark data frame named `medical_df`.  Drop any rows that contain NAN / Null values.  Check the schema and fix if needed.  Perform needed feature engineering using **only** a string indexer to get ready for training decision trees.  One hot encoding is not needed for random forest - do not use one hot encoding or any other transformations other than string indexing. 
- Split the data into variables named exactly train, test, and validation. Set the spark randomSplit seed argument to 2019.

In [6]:
# reading insurance data into spark dataframe
medical_df = spark.read.csv('insurance.csv', header=True, inferSchema=True)

# dropping any rows that contain NAN/Null values
medical_df = medical_df.dropna()

# feature engineering - converting categorical data (sex, smoker, and region) into numerical data using String Indexer
feature_encoding = feature.StringIndexer(inputCols=['sex', 'smoker', 'region'], 
                                         outputCols=['sex_indexed', 'smoker_indexed', 'region_indexed'])\
                                         .fit(medical_df)

medical_df = feature_encoding.transform(medical_df)

In [7]:
# splitting the data into train, test, and validation
train, test, validation = medical_df.randomSplit(weights=[0.65, 0.2, 0.15], seed=2019)
(train.count(), test.count(), validation.count())

(863, 255, 220)

In [8]:
#Print the schema
medical_df.printSchema()

#Print the shape
print('The shape of the dataframe is:', medical_df.toPandas().shape)

# print the head
medical_df.show()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)
 |-- sex_indexed: double (nullable = false)
 |-- smoker_indexed: double (nullable = false)
 |-- region_indexed: double (nullable = false)

The shape of the dataframe is: (1338, 10)
+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_indexed|smoker_indexed|region_indexed|
+---+------+------+--------+------+---------+-----------+-----------+--------------+--------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|        1.0|           1.0|           2.0|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|        0.0|           0.0|           0.0|
| 28|  male|  33.0|       3|    no|southeast|   4449

##### Grading Feedback Cell

The following questions will create a random forest regressor model.  The goal is to see if we can improve upon the linear regression score from homework 3. You can find the spark documentation for the random forest regressor [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-regression).

# Question 2 (10 pts)
Create and train a random forest regressor model using a grid search in the cell below.  Score your model using MSE.  Your grid search must be entirely encapsulated in the `if enable_grid_search` if statement.  The `enable_grid_search` Boolean is defined in a grading cell above.  You will disable the grid search before you submit by setting enable_grid_search to false.  Setting enable_grid_search to false should not result in a runtime error.  You will not receive full credit if any part of your grid search is outside of the if statement or if runtime errros result from setting the `enable_grid_search` variable to false.

In [9]:
if enable_grid_search:
  # training random forest model
  rf_pipe = Pipeline(stages=[feature.VectorAssembler(inputCols=['age', 'sex_indexed', 'bmi', 'children', 'smoker_indexed', 'region_indexed'], 
                                                     outputCol='rf_features'),
                             RandomForestRegressor(featuresCol='rf_features', labelCol='charges', seed=11272022)])
  
  grid = ParamGridBuilder().addGrid(rf_pipe.getStages()[-1].featureSubsetStrategy, ['sqrt', 'onethird', 'log2'])\
                            .addGrid(rf_pipe.getStages()[-1].numTrees, [10, 20, 45, 50, 55, 100, 500]).build()

  # building all the models
  all_models = []
  for gird_index in range(len(grid)):
    print('Fitting the model {}'.format(gird_index+1))
    rf_regressor = rf_pipe.fit(train, grid[gird_index])
    all_models.append(rf_regressor)
  
  rf_evaluator = RegressionEvaluator(predictionCol= rf_pipe.getStages()[-1].getPredictionCol(), 
                                     labelCol=rf_pipe.getStages()[-1].getLabelCol(), 
                                     metricName='mse')

  # evaluating the models
  mse_scores = [rf_evaluator.evaluate(rf_model.transform(test)) for rf_model in all_models]
  print(mse_scores)

  best_model_index = np.argmin(mse_scores)
  print("best model index =", best_model_index)

  print(grid[best_model_index].values())

##### Grading Feedback Cell

# Question 3 (10 pts)
Create a pipeline named `best_rf_pipe` that hard codes the tuning parameters from the best model found by the grid search in question 2 above.  Train and test best_rf_pipe.  Score your model using validation data and the MSE scoring metric.  Save train and validation MSE scores in variables named rf_train_mse and rf_validation_mse.

In [10]:
# hyperparameters
NUM_TREES = 50
NUM_FEATURES = 'sqrt'

# model building
best_rf_pipe = Pipeline(stages=[feature.VectorAssembler(inputCols=['age', 'sex_indexed', 'bmi', 'children', 'smoker_indexed', 'region_indexed'], 
                                                        outputCol='rf_features'),
                                RandomForestRegressor(featuresCol='rf_features', labelCol='charges',  seed=11272022, 
                                                      numTrees=NUM_TREES, featureSubsetStrategy=NUM_FEATURES)])
best_rf_model = best_rf_pipe.fit(train)

# model evaluation
best_rf_evaluator = RegressionEvaluator(predictionCol= best_rf_pipe.getStages()[-1].getPredictionCol(),
                                        labelCol=best_rf_pipe.getStages()[-1].getLabelCol(),
                                        metricName='mse')
rf_train_mse = np.round(best_rf_evaluator.evaluate(best_rf_model.transform(train)),2)
rf_validation_mse = np.round(best_rf_evaluator.evaluate(best_rf_model.transform(validation)),2)

In [11]:
# Grading cell do not modify
print("rf_train_mse =", rf_train_mse)
print("rf_validation_mse =", rf_validation_mse)

rf_train_mse = 16477454.74
rf_validation_mse = 14688149.02


##### Grading Feedback Cell
- Well done, these are low MSE scores as compared to others in the class.

# Question 4 (10 pts)
Use `best_rf_pipe` in question 3 for inference.  Create a pandas data frame named `rf_feature_importance` which contains 2 columns: `feature`, and `importance`.  Load the feature column with the feature name and the importance column with the feature importance score as determined by the random forest model. Sort the feature importances from high to low such that the most important feature is in the first row of the data frame.

In [12]:
rf_feature_importance = pd.DataFrame(data= list(zip(best_rf_pipe.getStages()[-2].getInputCols(),  
                                                    best_rf_model.stages[-1].featureImportances.toArray())), 
                                     columns=['feature', 'importance']).sort_values('importance', ascending=False)

In [13]:
# grading cell - do not modify
display(rf_feature_importance)

,feature,importance
4,smoker_indexed,0.735989
2,bmi,0.121727
0,age,0.117580
3,children,0.010180
5,region_indexed,0.009111
1,sex_indexed,0.005413


##### Grading Feedback Cell

# Question 5 (10 pts)
Repeat question 2 but this time use a [GBT regressor](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.GBTRegressor.html#pyspark.ml.regression.GBTRegressor).  Create and train a GBT regressor model using a grid search in the cell below.  Score your model using validation data and the MSE scoring metric.  Your grid search must be entirely encapsulated in the `if enable_grid_search` if statement.  The `enable_grid_search` Boolean is defined in a grading cell above.  You will disable the grid search before you submit by setting enable_grid_search to false.  Setting enable_grid_search to false should not result in a runtime error.  You will not receive full credit if any part of your grid search is outside of the if statement or if runtime errros result from setting the `enable_grid_search` variable to false.

In [14]:
if enable_grid_search:
  # training Gradient Booster model
  gb_pipe = Pipeline(stages=[feature.VectorAssembler(inputCols=['age', 'sex_indexed', 'bmi', 'children', 'smoker_indexed', 'region_indexed'], 
                                                     outputCol='gb_features'),
                             GBTRegressor(featuresCol='gb_features', labelCol='charges', seed=11272022)])
  
  gbt_grid = ParamGridBuilder().addGrid(gb_pipe.getStages()[-1].maxBins, [10,20,40])\
                                .addGrid(gb_pipe.getStages()[-1].maxDepth, [2,3,5])\
                                .addGrid(gb_pipe.getStages()[-1].maxIter, [10, 50, 100])\
                                .addGrid(gb_pipe.getStages()[-1].stepSize, [0.001, 0.002, 0.005, 0.01, 0.1]).build()

  # building all the models
  all_gbt_models = []
  for gbt_gird_index in range(len(gbt_grid)):
    print('Fitting GBT model {}'.format(gbt_gird_index+1))
    gbt_regressor = gb_pipe.fit(train, gbt_grid[gbt_gird_index])
    all_gbt_models.append(gbt_regressor)
  
  gb_evaluator = RegressionEvaluator(predictionCol= gb_pipe.getStages()[-1].getPredictionCol(), 
                                     labelCol=gb_pipe.getStages()[-1].getLabelCol(), 
                                     metricName='mse')

  # evaluating the models
  gbt_mse_scores = [gb_evaluator.evaluate(gbt_model.transform(test)) for gbt_model in all_gbt_models]
  print(gbt_mse_scores)

  best_gbt_model_index = np.argmin(gbt_mse_scores)
  print("best GBT model index =", best_gbt_model_index)

  print(gbt_grid[best_gbt_model_index].values())

##### Grading Feedback Cell

# Question 6 (10 pts Total)
This is a repeat of question 3 but for GBT.  Create a pipeline named `best_gbt_pipe` that hard codes the tuning parameters from the best model found by the grid search in question 5 above.  Train and test best_gbt_pipe using MSE as the scoring metric. Clearly print the resulting train and test MSE for `best_gbt_pipe` so it's easy for the graders to see your resulting MSEs.  Save train and test MSE scores in variables named gbt_train_mse and gbt_validation_mse.

In [15]:
# Hyperparameters
NUM_BINS = 10
MAX_DEPTH= 5
NUM_TREES = 50
LEARNING_RATE= 0.002

# model building
best_gbt_pipe = Pipeline(stages=[feature.VectorAssembler(inputCols=['age', 'sex_indexed', 'bmi', 'children', 'smoker_indexed', 'region_indexed'],
                                                          outputCol='gbt_features'),
                                 GBTRegressor(featuresCol='gbt_features',labelCol='charges', seed=11272022, 
                                              maxIter=NUM_TREES, stepSize=LEARNING_RATE, maxDepth=MAX_DEPTH, maxBins=NUM_BINS)])
best_gbt_model = best_gbt_pipe.fit(train)

# model evaluation
best_gbt_evaluator = RegressionEvaluator(predictionCol= best_gbt_pipe.getStages()[-1].getPredictionCol(),
                                         labelCol=best_gbt_pipe.getStages()[-1].getLabelCol(),
                                         metricName='mse')

gbt_train_mse = np.round(best_gbt_evaluator.evaluate(best_gbt_model.transform(train)),2)
gbt_validation_mse = np.round(best_gbt_evaluator.evaluate(best_gbt_model.transform(validation)),2)

In [16]:
print("gbt_train_mse =", gbt_train_mse)
print("gbt_validation_mse =", gbt_validation_mse)

gbt_train_mse = 16861956.02
gbt_validation_mse = 16082586.13


##### Grading Feedback Cell

# Question 7 (10 pts)
Create a pandas dataframe named `rf_gbt_mse_compare` which contains 3 columns: Model, Train MSE, and Validation MSE.  Load the Model column with "RF" or "GBT", the Train MSE column with the corresponding train MSE, and the Validation MSE column with the corresponding validation MSE scores from the random forest / gradient boosted tree scores.  Use rf_train_mse, rf_validation_mse, gbt_train_mse, and gbt_validation_mse variables to load the dataframe.  

GBT models usually produce better scores than random forest.  I am not sure if that will be the case for this dataset but you will be graded in comparison to other students in the class.

In [17]:
rf_gbt_mse_compare = pd.DataFrame(data=list(zip(['RF', 'GBT'],  [rf_train_mse, gbt_train_mse ], [rf_validation_mse, gbt_validation_mse])), 
                                  columns=['Model', 'Train MSE', 'Validation MSE'])

In [18]:
# Grading Cell Do Not Modify
display(rf_gbt_mse_compare)

,Model,Train MSE,Validation MSE
0,RF,16477454.74,14688149.02
1,GBT,16861956.02,16082586.13


##### Grading Feedback Cell
- -2 GBT should produce a lowere MSE as compared to best in class

# Question 8 (-5 pts if not performed)
Set the `enable_grid_search` Boolean variable to False in the grading cell at the top of this notebook.  Perform a __Runtime -> Disconnect and Delter Runtime__, __Runtime -> Run all__ test to verify there are no runtime errors.  Leave the `enable_grid_search` variable set to False and turn in your assignment.

# Question 9 (0 pts - get ready for part 2)
In part 2 of the assignment, we want to see how deep learning MSE scores compare to RF and GBT.  In the cell below, hard code variables which contain train and test scores for the models in part 1.

In [19]:
# Grading Cell - do not modify
# print the train and test scores
print("rf_train_mse =", rf_train_mse)
print("rf_validation_mse =", rf_validation_mse)
print("gbt_train_mse =", gbt_train_mse)
print("gbt_validation_mse =", gbt_validation_mse)

rf_train_mse = 16477454.74
rf_validation_mse = 14688149.02
gbt_train_mse = 16861956.02
gbt_validation_mse = 16082586.13


In [20]:
# uncomment and hard code the following variables using output from above.  
# You will copy this code for use in part 2 question 0
# hc_rf_train_mse = 16477454.74
# hc_rf_validation_mse = 14688149.02
# hc_gbt_train_mse = 16861956.02
# hc_gbt_validation_mse = 16082586.13